# Tensorflow Serving

Making tensorflow models available to others for use through API calls in productionproduction.

**This is part of TFX i.e Tensorflow Extended**

## Simple Hello World

For this we need to install certain things first:
- tensorflow-model-server (from AUR/apt/etc)
- tensorflow-serving-api (from pip)

In [5]:
import os
import numpy as np
import tensorflow as tf

### Create a basic Model

This is a simple model that calculate y=2x-1

In [3]:
# basic training data
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

# basic model
model = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])

model.compile(optimizer='sgd', loss='mean_squared_error')

history = model.fit(xs, ys, epochs=500, verbose=0)

print("Finished training the model")

Finished training the model


In [4]:
# testing the model
print(model.predict([10.0]))

[[18.985289]]


### Saving the model to be served

Here we try to save the model and later this saved model will be served through HTTP request and responses

In [21]:
model_dir = f'{os.getcwd()}/saved_model/hello_world/'
model_version = 1
model_export_path = os.path.join(model_dir, str(model_version))

if os.path.isdir(model_export_path):
    print('Model already exists...so replacing it with the latest')
    !rm -r {model_export_path}
    
model.save(model_export_path, save_format='tf')

print('Model saved....Here are the contents')
!ls -l {model_export_path}

INFO:tensorflow:Assets written to: /home/sbjr/my_workspace/tldr_tensorflow/v2/Advanced_Deployment/saved_model/hello_world/1/assets


INFO:tensorflow:Assets written to: /home/sbjr/my_workspace/tldr_tensorflow/v2/Advanced_Deployment/saved_model/hello_world/1/assets


Model saved....Here are the contents
total 48
drwxr-xr-x 2 sbjr sbjr  4096 Dec 18 20:53 assets
-rw-r--r-- 1 sbjr sbjr 39828 Dec 18 20:53 saved_model.pb
drwxr-xr-x 2 sbjr sbjr  4096 Dec 18 20:53 variables


### Examining the Model

In [22]:
!saved_model_cli show --dir {model_export_path} --all

2020-12-18 20:53:35.428855: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/lib64/:/opt/cuda-10.1/extras/CUPTI/lib64:/opt/cuda-10.1/lib64/:/opt/cudnn-7.6.5/lib64:/usr/lib64/:/opt/cuda-10.1/extras/CUPTI/lib64:/opt/cuda-10.1/lib64/:/opt/cudnn-7.6.5/lib64
2020-12-18 20:53:35.428886: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def[

### Run the Tensorflow Model Server

Here we will need to use the `signature_definition` value that we recieved from the `saved_model_cli` tool

In [23]:
os.environ['TF_MODEL_DIR']=model_export_path

In [24]:
os.environ['TF_MODEL_DIR']

'/home/sbjr/my_workspace/tldr_tensorflow/v2/Advanced_Deployment/saved_model/hello_world/1'

In [30]:
!which tensorflow_model_server

tensorflow_model_server not found


In [27]:
%%bash --bg
nohup tensorflow_model_server --rest_api_port=8505 --model_name=hello_world --model_base_path="${TF_MODEL_DIR}" > server.log 2>&1

In [28]:
!tail server.log

nohup: failed to run command 'tensorflow_model_server': No such file or directory


### Create data for testing API

In [ ]:
xs = np.array([[9.0], [10.0]])
data = json.dumps({"signature_name": "serving_default", "instances": xs.tolist()})
print(data)

### Send the Json Data to the API

In [ ]:
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/helloworld:predict', data=data, headers=headers)

print(json_response.text)

### Get the prediction

In [ ]:
predictions = json.loads(json_response.text)['predictions']
print(predictions)